In [3]:
##Calculating Piotroski's F score
import numpy as np
import pandas as pd

ImportError: No module named errors


Usage:   
  pip <command> [options]

no such option: --upgrade


In [1]:
##Profitability Ratios 4
roa=0; #return on assets
if(NI/TA>0):roa=1;

ocf=0; ##operational cash flow
if(CF/TA>0):ocf=1;

acc=0 ##accurals
if(roa-ocf<0):acc=1;

del_roa=0; #del_roa
if(roa[i-1]-roa[i]>0):del_roa=1;

#Capital Structure ratios 3
##change in leverage 
lev1=ltd/avg_lta; 
lev0=ltd0/avg_lta0;
del_lev=lev1-lev0;
if(del_lev<0):
    f_dellev=1;
else:
    f_dellev=0;

#change in liquidity
f_del_liquid=0; 
curr_rat1=curr_asset1/cur_liab1; ## Current ratio
curr_rat0=curr_asset0/cur_liab0;
del_liquid=curr_rat1-curr_rat0;
if(del_liquid>0): f_del_liquid=1;

eq_offer = 0; #if company has raised equity money in last one year put it one

##Efficiency Measures
f_del_margin=0; #del_margin
gmr1=g_margin/total_sales; ##gross margin ratio
gmr0=g_margin0/total_sales0;
del_margin= gmr1-gmr0;
if(del_margin>0):f_delmargin=1;

#del_turnover
f_del_turnover=0;
atr1=total_sales/total_asset_at_beg;
atr0=total_sales0/total_asset_at_beg0;
del_turnover=atr1-atr0;
if(del_turnover>0):f_del_turnover=1;


NameError: name 'NI' is not defined

In [ ]:
#Petroski's F_score
f_profitability=f_roa+f_ocf+f_accurals+f_del_roa;
f_capital=f_dellev+f_delliq+f_eq_offer;
f_efficiency=f_del_turnover+f_del_margin;

f_score= f_profitability+f_capital+f_efficiency